# What you can do with this notebook

By running this notebook, you can measure the exact execution time of your own and other participants' submissions.

## Import

In [ ]:
import datetime
import os
import time

import pandas as pd
from kaggle_secrets import UserSecretsClient

## Authentication


In [ ]:
COMPETITION =  'foursquare-location-matching'
KAGGLE_USERNAME = 'tubotubo' # Change to your username

user_secrets = UserSecretsClient()
os.environ["KAGGLE_USERNAME"] = KAGGLE_USERNAME
os.environ["KAGGLE_KEY"] = user_secrets.get_secret("kaggle_api") # Change to the key listed in kaggle.json


from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

## Main

In [ ]:
prev_lb_df = pd.DataFrame(api.competition_view_leaderboard(COMPETITION)["submissions"]).reset_index()
prev_lb_df["index"] = prev_lb_df["index"] + 1

while True:
    current_lb_df = pd.DataFrame(api.competition_view_leaderboard(COMPETITION)["submissions"]).reset_index()
    current_lb_df["index"] = current_lb_df["index"] + 1

    merge_df = current_lb_df.merge(prev_lb_df, how="left", on="teamId").fillna(0)
    merge_df["runtime"] = (
        datetime.datetime.now(datetime.timezone.utc) - pd.to_datetime(merge_df["submissionDate_x"])
    ).dt.seconds / 60
    merge_df["score_change"] = merge_df["score_x"].astype(float) - merge_df["score_y"].astype(float)
    merge_df = merge_df.query("score_x != score_y")

    for _, row in merge_df.iterrows():
        print(datetime.datetime.now(datetime.timezone.utc))
        print('')
        print(f'Score has changed! {row["score_x"]} (+{row["score_change"]})')
        print(f'Ranking: {row["index_y"]} -> {row["index_x"]}')
        print(f'Runtime[m]: {row["runtime"]}')
        print(f'@{row["teamNameNullable_x"]}')
        print("--" * 10)

    prev_lb_df = current_lb_df

    time.sleep(60)
    
'''
output example

2022-06-17 17:58:51.997861+00:00

Score has changed! 0.866 (+0.005)
Ranking: 96 -> 80
Runtime[m]: 120
@Kuruton
'''